In [1]:
import numpy as np
from scipy.io import loadmat
from sklearn.neighbors import NearestNeighbors

In [2]:
# Load all of the saved files
train_feat = np.loadtxt('train_feat.gzip', dtype = 'float')
gallery_feat = np.loadtxt('gallery_feat.gzip', dtype = 'float')
query_feat = np.loadtxt('query_feat.gzip', dtype = 'float')
val_feat = np.loadtxt('val_feat.gzip', dtype = 'float')
train_noval_feat = np.loadtxt('train_noval_feat.gzip', dtype = 'float')
train_labels = np.loadtxt('train_labels.gzip', dtype = 'uint16')
gallery_labels = np.loadtxt('gallery_labels.gzip', dtype = 'uint16')
query_labels = np.loadtxt('query_labels.gzip', dtype = 'uint16')
val_labels = np.loadtxt('val_labels.gzip', dtype = 'uint16')
train_noval_labels = np.loadtxt('train_noval_labels.gzip', dtype = 'uint16')
#train_filelist = np.loadtxt('train_filelist.gzip', dtype = 'uint16')
#gallery_filelist = np.loadtxt('gallery_filelist.gzip', dtype = 'uint16')
#query_filelist = np.loadtxt('query_filelist.gzip', dtype = 'uint16')
#val_filelist = np.loadtxt('val_filelist.gzip', dtype = 'uint16')
#train_noval_filelist = np.loadtxt('train_noval_filelist.gzip', dtype = 'uint16')
train_camId = np.loadtxt('train_camId.gzip', dtype = 'uint16') # uint8
gallery_camId = np.loadtxt('gallery_camId.gzip', dtype = 'uint16')
query_camId = np.loadtxt('query_camId.gzip', dtype = 'uint16')
val_camId = np.loadtxt('val_camId.gzip', dtype = 'uint16')
train_noval_camId = np.loadtxt('train_noval_camId.gzip', dtype = 'uint16')

In [3]:
# Function that takes as input an index that refers to a query image
# and returns the gallery without the unnecessary pictures in it
# Remove the pictures from the gallery that have the same label and are taken by the same camera as out QUERY[index]
# picture in the query array
def get_removed_gallery (index):
    query_camId_index = np.ones((gallery_labels.shape))*query_camId[index] # This vector is for comparison, has all of the elements as the query camID.
    gallery_label_index = np.ones((gallery_labels.shape))*gallery_labels[index] # This vector stores the first item's gallery label
    selected_indices = np.logical_not(np.logical_and(gallery_camId == query_camId_index, gallery_labels == gallery_label_index))
    return gallery_feat[selected_indices, :], gallery_labels[selected_indices]

In [4]:
a, b = get_removed_gallery(0)
print (a, a.shape, b, b.shape)

[[0.76888168 0.25586957 0.60124129 ... 0.18115909 0.43165341 0.09931619]
 [1.39672875 0.77182716 0.70401114 ... 0.01811628 0.08462342 0.12913167]
 [1.30696166 0.22219124 0.62375975 ... 0.02954952 0.37348923 0.05303315]
 ...
 [0.01961832 0.17617127 0.54575312 ... 0.33026204 0.93925601 0.56220758]
 [0.05719076 0.60635394 0.58300596 ... 0.40994745 1.37061644 0.55975795]
 [0.07245111 0.51656747 0.59082419 ... 0.60488421 1.05610693 0.72939718]] (5325, 2048) [   3    3    3 ... 1463 1463 1463] (5325,)


In [15]:
# k nearest neighbours implementation
k = 1
print (query_labels.shape)
accuracies = np.zeros((query_labels.shape[0]))
print (accuracies, accuracies.shape)
for i in range (0, query_labels.shape[0], 1):
    gallery_feat_removed, gallery_labels_removed = get_removed_gallery (i)
    nbrs = NearestNeighbors(n_neighbors=k).fit(gallery_feat_removed)
    distances, indices = nbrs.kneighbors(query_feat[i:i+1, :])
    print(gallery_labels_removed[indices], query_labels[i])
    is_same_label = (np.ones((k))*query_labels[i] == gallery_labels_removed[indices])
    accuracy = np.sum(is_same_label)/k
    print (accuracy)
    accuracies[i]=accuracy
    
print (accuracies.mean())

(1400,)
[0. 0. 0. ... 0. 0. 0.] (1400,)
[[1368]] 3
0.0
[[517]] 3
0.0
[[6]] 6
1.0
[[6]] 6
1.0
[[7]] 7
1.0
[[7]] 7
1.0
[[11]] 11
1.0
[[11]] 11
1.0
[[14]] 14
1.0
[[14]] 14
1.0
[[17]] 17
1.0
[[17]] 17
1.0
[[22]] 22
1.0
[[22]] 22
1.0
[[28]] 28
1.0
[[28]] 28
1.0
[[30]] 30
1.0
[[30]] 30
1.0
[[32]] 32
1.0
[[32]] 32
1.0
[[33]] 33
1.0
[[33]] 33
1.0
[[34]] 34
1.0
[[34]] 34
1.0
[[35]] 35
1.0
[[35]] 35
1.0
[[38]] 38
1.0
[[38]] 38
1.0
[[40]] 40
1.0
[[40]] 40
1.0
[[41]] 41
1.0
[[41]] 41
1.0
[[42]] 42
1.0
[[42]] 42
1.0
[[45]] 45
1.0
[[45]] 45
1.0
[[47]] 47
1.0
[[47]] 47
1.0
[[51]] 51
1.0
[[51]] 51
1.0
[[54]] 54
1.0
[[54]] 54
1.0
[[55]] 55
1.0
[[55]] 55
1.0
[[58]] 58
1.0
[[58]] 58
1.0
[[60]] 60
1.0
[[60]] 60
1.0
[[62]] 62
1.0
[[62]] 62
1.0
[[68]] 68
1.0
[[68]] 68
1.0
[[69]] 69
1.0
[[69]] 69
1.0
[[70]] 70
1.0
[[70]] 70
1.0
[[72]] 72
1.0
[[72]] 72
1.0
[[74]] 74
1.0
[[74]] 74
1.0
[[76]] 76
1.0
[[76]] 76
1.0
[[77]] 77
1.0
[[77]] 77
1.0
[[78]] 78
1.0
[[78]] 78
1.0
[[80]] 80
1.0
[[80]] 80
1.0
[[81]] 81
1.0
[

[[529]] 529
1.0
[[530]] 530
1.0
[[530]] 530
1.0
[[532]] 532
1.0
[[532]] 532
1.0
[[536]] 536
1.0
[[536]] 536
1.0
[[537]] 537
1.0
[[537]] 537
1.0
[[541]] 541
1.0
[[541]] 541
1.0
[[542]] 542
1.0
[[542]] 542
1.0
[[544]] 544
1.0
[[544]] 544
1.0
[[545]] 545
1.0
[[545]] 545
1.0
[[547]] 547
1.0
[[547]] 547
1.0
[[548]] 548
1.0
[[548]] 548
1.0
[[550]] 550
1.0
[[550]] 550
1.0
[[552]] 552
1.0
[[552]] 552
1.0
[[555]] 555
1.0
[[555]] 555
1.0
[[556]] 556
1.0
[[556]] 556
1.0
[[557]] 557
1.0
[[557]] 557
1.0
[[558]] 558
1.0
[[558]] 558
1.0
[[559]] 559
1.0
[[559]] 559
1.0
[[560]] 560
1.0
[[560]] 560
1.0
[[562]] 562
1.0
[[562]] 562
1.0
[[564]] 564
1.0
[[564]] 564
1.0
[[566]] 566
1.0
[[566]] 566
1.0
[[569]] 569
1.0
[[569]] 569
1.0
[[570]] 570
1.0
[[570]] 570
1.0
[[573]] 573
1.0
[[573]] 573
1.0
[[575]] 575
1.0
[[575]] 575
1.0
[[578]] 578
1.0
[[578]] 578
1.0
[[584]] 584
1.0
[[584]] 584
1.0
[[585]] 585
1.0
[[585]] 585
1.0
[[586]] 586
1.0
[[586]] 586
1.0
[[590]] 590
1.0
[[590]] 590
1.0
[[592]] 592
1.0
[[321]] 

[[1028]] 1028
1.0
[[1030]] 1030
1.0
[[1030]] 1030
1.0
[[1032]] 1032
1.0
[[1032]] 1032
1.0
[[1037]] 1037
1.0
[[1037]] 1037
1.0
[[1041]] 1041
1.0
[[1041]] 1041
1.0
[[1042]] 1042
1.0
[[1042]] 1042
1.0
[[1043]] 1043
1.0
[[1043]] 1043
1.0
[[1044]] 1044
1.0
[[1044]] 1044
1.0
[[1047]] 1047
1.0
[[1047]] 1047
1.0
[[1049]] 1049
1.0
[[1049]] 1049
1.0
[[912]] 1052
0.0
[[1052]] 1052
1.0
[[1054]] 1054
1.0
[[1054]] 1054
1.0
[[1061]] 1061
1.0
[[1061]] 1061
1.0
[[1062]] 1062
1.0
[[1062]] 1062
1.0
[[1063]] 1063
1.0
[[1063]] 1063
1.0
[[1064]] 1064
1.0
[[1017]] 1064
0.0
[[1065]] 1065
1.0
[[1065]] 1065
1.0
[[1068]] 1068
1.0
[[1068]] 1068
1.0
[[925]] 1069
0.0
[[1069]] 1069
1.0
[[1075]] 1075
1.0
[[1075]] 1075
1.0
[[1077]] 1077
1.0
[[1077]] 1077
1.0
[[1078]] 1078
1.0
[[1078]] 1078
1.0
[[1081]] 1081
1.0
[[1081]] 1081
1.0
[[922]] 1088
0.0
[[1088]] 1088
1.0
[[1091]] 1091
1.0
[[1091]] 1091
1.0
[[1093]] 1093
1.0
[[1093]] 1093
1.0
[[1095]] 1095
1.0
[[1095]] 1095
1.0
[[1096]] 1096
1.0
[[1096]] 1096
1.0
[[1097]] 1097

In [ ]:



well_predicted = gallery_labels_removed[indices, :] == np.ones((k))*query_labels[0]
print (well_predicted, well_predicted.shape)
precision_or_recall = np.sum(well_predicted)/k
print (precision_or_recall)

In [ ]:
print (query_feat.dtype, query_feat.max())
print (gallery_feat, gallery_feat.shape)
print (query_feat, query_feat.shape)

In [ ]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors([[0, 1], [0,0]])
print(indices)
print (distances)

In [ ]:
k = 5
nbrs = NearestNeighbors(n_neighbors=k).fit(gallery_feat)
distances, indices = nbrs.kneighbors(query_feat)
print(indices, indices.shape)
print (distances, distances.shape)